In [1]:
from pathlib import Path
import random
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
from tabulate import tabulate
from omnibelt import human_readable_number, load_json, save_json
import omnifig as fig
from sink import database, misc
# from sink.processing import recursive_collect_dupes
import humanize
import pandas as pd
fig.initialize()

In [2]:
db = fig.create_config(_type='file-db', db_path=misc.data_root() / 'largetest.db').pull(silent=True)
db

In [3]:
base = Path(r'D:\Trust')
base.exists()

True

In [4]:
# full_raw = load_json(misc.data_root() / 'kill.json')
# klist = full_raw['targets']
# clusters = full_raw['clusters']
# reds = [Path(r) for r in full_raw['reds']]
# len(klist), len(clusters), len(reds)

In [5]:
groups = [[Path(p) for p in ps] for ps in load_json(misc.data_root() / 'candidates.json')]
cands = [p for group in groups for p in group]
infos = {}
for path in tqdm(cands):
	code, (size, modtime) = db.find_path(path)
	infos[path] = {'size': size, 'modtime': datetime.fromtimestamp(modtime), 'code': code}
# for path in tqdm(reds):
# 	if path not in infos:
# 		code, (size, modtime) = db.find_path(path)
# 		infos[path] = {'size': size, 'modtime': datetime.fromtimestamp(modtime), 'code': code}
len(cands), len(groups), len(infos)

  0%|          | 0/44916 [00:00<?, ?it/s]

(44916, 15760, 44916)

In [6]:
def get_info(path):
	if path not in infos:
		code, (size, modtime) = db.find_path(path)
		infos[path] = {'size': size, 'modtime': datetime.fromtimestamp(modtime), 'code': code}
	return infos[path]
def rank_path(path):
	return 'old' in str(path).lower(), len(path.parents), len(path.name), len(str(path)), path.name

In [7]:
codes = {}
for path in tqdm(cands):
	codes.setdefault(infos[path]['code'], []).append(path)
	
codes = {code: cluster for code, cluster in codes.items() if any(p.is_file() for p in cluster) and any(p.is_dir() for p in cluster)}

for code, cluster in codes.items():
	# if any(p.is_file() for p in cluster) and any(p.is_dir() for p in cluster):
	# 	cluster = [next(p.iterdir()) if p.is_dir() and len(list(p.iterdir())) == 1 else p for p in cluster]
	codes[code] = sorted(cluster, key=rank_path)

len(codes)

  0%|          | 0/44916 [00:00<?, ?it/s]

111

In [8]:
print(tabulate([
	[
		len(cluster),
		get_info(cluster[0])['code'][:6],
		humanize.naturalsize(get_info(cluster[0])['size']),
	 # p.relative_to(base)
		'\n'.join(get_info(p)['modtime'].strftime('%Y-%m-%d %H:%M:%S') for p in cluster),
		 '\n'.join(str(p) for p in cluster), 
		#p.relative_to(base),
		# f'{p.name}\n{p.relative_to(base)}', 
	 ] 
	for code, cluster in sorted(codes.items(), key=lambda item: (get_info(item[1][0])['size'], len(item[1])), reverse=True)[:100]]))
	# for code, cluster in sorted(codes.items(), key=lambda item: infos[item[1][0]]['size'], reverse=True)[:100]], headers=['code', 'size', 'modtime', 'path']))

--  ------  ---------  -------------------  -------------------------------------------------------------------------------------------------------------------
 2  b494bb  360.1 MB   2020-02-11 23:43:05  D:\Trust\Music\Audio Books\Superintelligence Paths Danges Strategies
                       2015-12-25 09:57:43  D:\Trust\Stuff\Superintelligence\Superintelligence_ Paths, Dangers, Strategies (Unabridged).m4a
 3  22db64  142.5 MB   2020-02-12 17:32:20  D:\Trust\Files\Extra Onedrive\Shared\cba\cultural slides
                       2013-06-03 06:28:40  D:\Trust\Files\Khan 2013\World History 2\cba\finals\Eastern European Culture Returns.pptx
                       2013-07-24 10:30:42  D:\Trust\Files\Skydrive 2013-2014\Old\Eastern European Culture Returns.pptx
 2  f35fdf  81.6 MB    2011-04-01 19:41:04  D:\Trust\Old\Orion\Videos\Final Product Part 1.mp4
                       2020-02-12 00:11:26  D:\Trust\Old\Orion\Videos\New folder (2)\ORION 4-1-2011 10.32.11 AM\Canceled Encodes
 2  9748

In [9]:
# print(tabulate([
# 	[
# 	  info['code'][:6], 
# 	 humanize.naturalsize(info['size']), 
# 	  info['modtime'].strftime('%Y-%m-%d %H:%M:%S'), 
# 	 # p.relative_to(base)
# 		 p, #p.relative_to(base),
# 		# f'{p.name}\n{p.relative_to(base)}', 
# 	 ] 
# 	for p, info in sorted(infos.items(), key=lambda item: item[1]['size'], reverse=True)[:100]], headers=['code', 'size', 'modtime', 'path']))

In [10]:
bad_key = '00000000000000000000000000000000'

In [18]:
hits = [{'path': path, 'size': size, 'modtime': datetime.fromtimestamp(modtime)} for path, (size, modtime) in db.find_duplicates(bad_key)]
hits = sorted(hits, key=lambda info: info['size'], reverse=True)
len(hits)

38

In [17]:
print(tabulate([(info['size'], info['modtime'].strftime('%Y-%m-%d %H:%M:%S'), info['path']) for info in hits]))

--------  -------------------  -------------------------------------------------------------------------------------------------------------------------------
21389774  2020-02-12 17:40:00  D:\Trust\Files\OneDrive 2015-2016\Spring\Physical Chemistry III
 2350588  2020-02-12 17:45:48  D:\Trust\Files\Research\AIResearch\pose_rl\se3nets-pytorch\cartpole\saves
 2350588  2020-02-12 17:46:02  D:\Trust\Files\Research\AIResearch\se3_control\cartpole\ddpg_pendulum\saves
   86000  2020-02-12 18:05:04  D:\Trust\Files\Skydrive 2014-2015\Other\nand2tetris\tools\builtInChips
   32768  2020-02-12 17:32:54  D:\Trust\Files\Khan 2012\Other\Games\.Minecraft usb\data\Microsoft\Windows\IETldCache
   32768  2020-02-12 01:39:58  D:\Trust\Stuff\Games\.Minecraft usb\data\Microsoft\Windows\IETldCache
   28512  2020-02-12 17:45:18  D:\Trust\Files\Research\AIResearch\foundation\results\old\test-walking\Jun-04-2018-184914
   28512  2020-02-12 17:45:20  D:\Trust\Files\Research\AIResearch\foundation\results\test-wal

In [22]:
path = hits[0]['path']
print(path)

D:\Trust\Files\OneDrive 2015-2016\Spring\Physical Chemistry III


In [23]:
db.process_dir(path)

(WindowsPath('D:/Trust/Files/OneDrive 2015-2016/Spring/Physical Chemistry III'),
 ('00000000000000000000000000000000', (21389774, 1581525600.44344)))

In [25]:
hcodes = []
contents = []
for content_path in path.iterdir():
	info = db.find_path(content_path)
	hcodes.append(info[0])
	if info is None:
		raise ValueError(f'Missing path: {content_path}')
	contents.append(info)
len(contents)

31

In [27]:
misc.xor_hexdigests(hcodes[0], hcodes[1])

'daf43079abb9ecceb45d79eb8bbc697b'

In [30]:
progress = []
c = hcodes[0]
for h in hcodes[1:]:
	c = misc.xor_hexdigests(c, h)
	progress.append(c)
progress#[-1]

['daf43079abb9ecceb45d79eb8bbc697b',
 '19af21b69c4a2efc980cff1745d02827',
 'c2dda6b796766ef84890042e50972dff',
 '30070f29fa03db17f14b9b4cf0958677',
 '2a5b309f667e8bd68e20903f370f26d9',
 '10679555182ce40db95bf300c2f8e2c5',
 '2a5b309f667e8bd68e20903f370f26d9',
 '1554f5249ef10b2f1884f53f336c3c60',
 '29d81099ec40ffcc1df1e9db0f6a9a92',
 '7450bce3b03916a2412fe9296c7f52bd',
 '29d81099ec40ffcc1df1e9db0f6a9a92',
 'ccee03397c35ac5042156b9e72f4b3d4',
 '29d81099ec40ffcc1df1e9db0f6a9a92',
 '66d48a88835b1525f1469b34421be763',
 '50d92444e19426b5ee9f6bd433bd3667',
 '13ab15a42adcc59307101d03dbaccf9d',
 '07fd5da9e3727956533463ef7d8e0316',
 '36d0ef8b3918c54fe4f98bb62ba77afa',
 'ea3196e80851115cf4589fc734b9ea9e',
 'd2b434f131f3f24b2c872cbfdf1c7648',
 'a93e913dc5258eb864490868e9a73ae0',
 '6f3fba7602cd6807be9d1c1420fd0455',
 '09b3ae513020aa4cc17cdff79fc2e202',
 '8d269cd4ec0757a7becf3c12163f2728',
 '91f7d8ad0977a00a46552665b1c107b8',
 '488c205f95f7fed083216cb858c72cc1',
 '4037295bc2366700ae01b9ab31410168',
 

In [31]:
hcodes[-1]

'2a7f209558cb2096b4c7a1e1e2ad0257'

In [32]:
from collections import Counter

In [33]:
Counter(hcodes)

Counter({'3a3ca5ca7e526fdb377b633ff5f7c41c': 2,
         '5d88ac7a5c79e96e5cde00f26315c82f': 2,
         'e53613a09075539c5fe482457d9e2946': 2,
         '7495dd3e7fb7fa2f2436fe244c64cffd': 1,
         'ae61ed47d40e16e1906b87cfc7d8a686': 1,
         'c35b11cf37f3c2322c5186fcce6c415c': 1,
         'db7287010a3c4004d09cfb39154705d8': 1,
         'f2daa99e6c75b5efb9db9f62a002ab88': 1,
         '1a5c3fb69c7d50c17f6b0b73c79aa0ae': 1,
         '3f0fc5bbf88f80f996a4650004631ab9': 1,
         '3c8ce5bd72b1f4e305751ce43c06a6f2': 1,
         '4f0c9a116f1beae9ecb772ef4d717df1': 1,
         '360daecc62cf33901fd9f0e071a6d104': 1,
         '437231e0cb48e326e98f76d7e811f9fa': 1,
         '1456480dc9aebcc554247eeca622cc8b': 1,
         '312db222da6abc19b7cde859562979ec': 1,
         'dce179633149d41310a114711f1e9064': 1,
         '3885a21939a2e317d8dfb378eba59cd6': 1,
         '7b8aa5ccf4d67cf348ce24d736bb4ca8': 1,
         'c6012b4bc7e8e6bfdad4147cc95a3eb5': 1,
         '668c142732edc24b7fe1c3e3bf3fe6

In [54]:
data = b''.join(bytes.fromhex(code) for code in sorted([]))

In [55]:
data

b''

In [38]:
ec = misc.md5_hash(b'')
ec

'd41d8cd98f00b204e9800998ecf8427e'

In [42]:
misc.md5_hash(bytes.fromhex(ec))

'59adb24ef3cdbe0297f05b395827453f'

In [44]:
len(bytes.fromhex(ec))

16